In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/Shareddrives/Unlimited/Colab/AlphaVision/"
MODEL = "densenet161_Kfold"
model_path = "models_" + MODEL + "/"

In [ ]:
from google.colab import output
# !cp 파일1 파일2 # 파일1을 파일2로 복사 붙여넣기
!cp "/content/drive/Shareddrives/Unlimited/Colab/AlphaVision/AlphaVision.zip" "data_2.zip"
# data_2.zip을 현재 디렉터리에 압축해제
!unzip "data_2.zip"

^C
Archive:  data_2.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of data_2.zip or
        data_2.zip.zip, and cannot find data_2.zip.ZIP, period.


In [ ]:
from google.colab import output
# 현재 디렉터리에 dirty_mnist라는 폴더 생성
!mkdir "./dirty_mnist"
#dirty_mnist.zip라는 zip파일을 dirty_mnist라는 폴더에 압축 풀기
!unzip "dirty_mnist_2nd.zip" -d "./dirty_mnist/"
# 현재 디렉터리에 test_dirty_mnist라는 폴더 생성
!mkdir "./test_dirty_mnist"
#test_dirty_mnist.zip라는 zip파일을 test_dirty_mnist라는 폴더에 압축 풀기
!unzip "test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist/"
# 출력 결과 지우기
output.clear()

In [ ]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
#from torchinfo import summary

from torchvision import transforms
from torchvision.models import resnet50

## 1. 커스텀 데이터셋 만들기

In [ ]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

## 2. 이미지 어그멘테이션

In [ ]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [ ]:
trainset = MnistDataset('dirty_mnist', 'dirty_mnist_2nd_answer.csv', transforms_train)
testset = MnistDataset('test_dirty_mnist', 'sample_submission.csv', transforms_test)

train_loader = DataLoader(trainset, batch_size=64, num_workers=8)
test_loader = DataLoader(testset, batch_size=32, num_workers=4)

## 3. ResNet50 모형

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Mon Feb 22 17:52:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    38W / 300W |  16125MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, 26)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel().to(device)
#print(summary(model, input_size=(1, 3, 256, 256), verbose=0))

## 4. 학습하기

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()

num_epochs = 10
model.train()

for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            print(f'{epoch}: {loss.item():.5f}, {acc.item():.5f}')

0: 0.68991, 0.54567
0: 0.69040, 0.53726
0: 0.68866, 0.53185
0: 0.68997, 0.52945
0: 0.68796, 0.53846
0: 0.68793, 0.53546
0: 0.68440, 0.53906
1: 0.67826, 0.54507
1: 0.68016, 0.54627
1: 0.68054, 0.54207
1: 0.67940, 0.54387
1: 0.68377, 0.54447
1: 0.66449, 0.55529
1: 0.66938, 0.55469
2: 0.64759, 0.57873
2: 0.66110, 0.57332
2: 0.65042, 0.56971
2: 0.66341, 0.56310
2: 0.65235, 0.58474
2: 0.63722, 0.58894
2: 0.64674, 0.58353
3: 0.62213, 0.61358
3: 0.63675, 0.61118
3: 0.62265, 0.60817
3: 0.63886, 0.60637
3: 0.62793, 0.61118
3: 0.60944, 0.62320
3: 0.62069, 0.61959
4: 0.58665, 0.64663
4: 0.62409, 0.61839
4: 0.59197, 0.64363
4: 0.61009, 0.64243
4: 0.60083, 0.64663
4: 0.57773, 0.65565
4: 0.58763, 0.65024
5: 0.54660, 0.69291
5: 0.57810, 0.66346
5: 0.55304, 0.68510
5: 0.56184, 0.68089
5: 0.55741, 0.68450
5: 0.54610, 0.69892
5: 0.56174, 0.69231
6: 0.49047, 0.73858
6: 0.53025, 0.72055
6: 0.50879, 0.72596
6: 0.53637, 0.70913
6: 0.52099, 0.72776
6: 0.51725, 0.73377
6: 0.52959, 0.72837
7: 0.44324, 0.78305


## 5. 추론하기

In [ ]:
submit = pd.read_csv('sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()
    
submit.to_csv('submit.csv', index=False)